In [ ]:
import tensorflow as tf
import csv
import pickle
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import numpy as np
import pandas as pd

tf.logging.set_verbosity(tf.logging.ERROR)
validation_path = r"..\data\validation.csv"
test_path = r"C:..\data\test.csv"
output_path = r'..\results\test_bidprices.csv'
DL_validation_data_path = r"..\data\validation_data.p"
DL_validation_payprice_labels_path = r"..\data\validation_labels_continuous.p"
DL_test_data_path = r"..\data\test_data.p"
DL_test_labels_path = r"..\data\test_labels_continuous.p"
validation_targets_path = r"..\data\validation_targets.p"
model_dir_1 = r"..\model"
ctr_training_data_path = r"..\data\ctr_training_data.p"
ctr_training_labels_path = r"..\data\ctr_training_labels.p"
ctr_validation_data_path = r"..\data\ctr_validation_data.p"
ctr_test_data_path = r"..\data\ctr_test_data.p"
ctr_test_labels_path = r"..\data\ctr_test_labels.p"
neg_weight = 0.0007539649884458758
pos_weight = 0.9992460350115542

vectorizer = DictVectorizer()
label_encoder = LabelEncoder()


COLUMNS = ['weekday','hour','region','city','adexchange','advertiser','os','browser','usertag']
LABEL_COLUMN = "payprice"
CATEGORICAL_COLUMNS = ['weekday','hour','region','city','adexchange','advertiser','os','browser','usertag']
WEIGHT_COLUMN = 'weight'
USERTAGS = ['13776', '10133', '10146', '10052', '13800', '13678', '10077', '10057', '10048',
            '16753', '16706', '10120', '11278', '10140', '10127', '10684', '10138', '10148', '11092',
            '15398', '10067', '11632', '10117', '10114', '10145', '11576', '14273', '10059', '16617',
            '10083', '13403', '10126', '11944', '13874', '11724', '10076', '10131', '10093', '11423',
            '10110', '10123', '16751', '13496', '10149', '10111', '10031', '10142', '10118', '10074',
            '10024', '16593', '10006', '10116', '11680', '10130', '10147', '10102', '10063',
            '10075', '11512', '10129', '10079', '10125', '10115', '13042', '11379', '16661', '13866']

dummy_data = ({'os': ['windows'], 'weekday': ['1'], 'browser': ['ie'], 
              'usertag': ['00000000000000000000000000000000000000000000000000000000010000000001'], 
              'weight': [0.0007539649884458758], 'city': ['85'], 'advertiser': ['1458'], 'region': ['80'],
              'adexchange': ['2'], 'hour': ['14']},102)

dnn_hidden_layers_param = [100,50]

In [ ]:
#Logistic regression training
def lr_train(data:list,labels:list):                
    train_event_x = vectorizer.fit_transform(data)
    train_event_y = label_encoder.fit_transform(labels)

    p = 0.13
    lr = LogisticRegression(C=p, class_weight={1: pos_weight, 0: neg_weight})
    lr.fit(train_event_x, train_event_y)
    
    return lr

def lr_evaluation(model,data):
    event_x = vectorizer.transform(data)
    event_y = label_encoder.inverse_transform(model.predict(event_x))
    #event_y = model.predict_proba(event_x)
    #print(event_y)
    return event_y

def load_data(path):
    return pickle.load(open(path,"rb"))

In [ ]:
def restore_model(model_dir, dummy_data, model_type=None):
    weekday = tf.contrib.layers.sparse_column_with_keys(column_name="weekday",
                                                     keys=["0","1","2","3","4","5","6"],
                                                     combiner="sqrtn")
    hour = tf.contrib.layers.sparse_column_with_keys(column_name="hour",
                                                     keys=["0","1","2","3","4","5","6","7"
                                                          "8","9","10","11","12","13","14",
                                                           "15","16","17","18","19","20","21",
                                                           "22","23"],
                                                    combiner="sqrtn")
    region = tf.contrib.layers.sparse_column_with_hash_bucket(
      "region", hash_bucket_size=100,combiner="sqrtn")
    city = tf.contrib.layers.sparse_column_with_hash_bucket(
      "city", hash_bucket_size=1000,combiner="sqrtn")
    adexchange = tf.contrib.layers.sparse_column_with_keys(column_name="adexchange",
                                                     keys=["1","2","3","4","null"],
                                                          combiner="sqrtn")
    advertiser = tf.contrib.layers.sparse_column_with_hash_bucket(
      "advertiser", hash_bucket_size=20,combiner="sqrtn")
    os = tf.contrib.layers.sparse_column_with_hash_bucket(
      "os", hash_bucket_size=100,combiner="sqrtn")
    browser = tf.contrib.layers.sparse_column_with_hash_bucket(
      "browser", hash_bucket_size=100,combiner="sqrtn")
    usertag = tf.contrib.layers.sparse_column_with_hash_bucket(
      "usertag", hash_bucket_size=1000000,combiner="sqrtn")
    
    # Wide columns and deep columns.
    wide_columns = [weekday,hour,region,city,adexchange,advertiser,os,browser,usertag]

    deep_columns = [
      tf.contrib.layers.embedding_column(weekday, dimension=4,combiner="sqrtn"),
      tf.contrib.layers.embedding_column(hour, dimension=4,combiner="sqrtn"),
      tf.contrib.layers.embedding_column(region, dimension=4,combiner="sqrtn"),
      tf.contrib.layers.embedding_column(city,dimension=4,combiner="sqrtn"),
      tf.contrib.layers.embedding_column(adexchange, dimension=4,combiner="sqrtn"),
      tf.contrib.layers.embedding_column(advertiser, dimension=4,combiner="sqrtn"),
      tf.contrib.layers.embedding_column(os, dimension=4,combiner="sqrtn"),
      tf.contrib.layers.embedding_column(browser, dimension=4,combiner="sqrtn"),
      tf.contrib.layers.embedding_column(usertag, dimension=8,combiner="sqrtn"),
      ]
    
    if model_type == 'CTR_estimator':
        weight_column = tf.contrib.layers.real_valued_column("weight")
        
        model = tf.contrib.learn.DNNLinearCombinedClassifier(
            # wide settings
            linear_feature_columns=wide_columns,
            linear_optimizer=tf.train.FtrlOptimizer(learning_rate=0.1,
                                                l1_regularization_strength=0.001,
                                                l2_regularization_strength=0.001),
            # deep settings
            dnn_feature_columns=deep_columns,
            #dnn_hidden_units=[1000, 500, 100],
            dnn_hidden_units=[100,50],
            dnn_optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.1,
                                                        l1_regularization_strength=0.001,
                                                        l2_regularization_strength=0.001),
            config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1800),
            n_classes=31,
            model_dir=model_dir,
            weight_column_name = WEIGHT_COLUMN
        )
        
        model.fit(input_fn=lambda: input_fn(dummy_data[0],dummy_data[2],model_type), steps=1)
    
    else:
        
        model = tf.contrib.learn.DNNLinearCombinedRegressor(
            # wide settings
            linear_feature_columns=wide_columns,
            linear_optimizer=tf.train.FtrlOptimizer(learning_rate=0.1,
                                                    l1_regularization_strength=0.001,
                                                    l2_regularization_strength=0.001),
            #deep settings
            dnn_feature_columns=deep_columns,
            #dnn_hidden_units=[1000, 500, 100],
            dnn_hidden_units=dnn_hidden_layers_param,
            dnn_optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.1,
                                                            l1_regularization_strength=0.001,
                                                            l2_regularization_strength=0.001),
            config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1800),
            model_dir=model_dir
        )
        
        model.fit(input_fn=lambda: input_fn(dummy_data[0],[dummy_data[1]]), steps=1)
    
    return model

def input_fn(data,labels, first=None, last=None):
    categorical_cols = {}
    for k in CATEGORICAL_COLUMNS:
        categorical_cols[k] = tf.SparseTensor(
            indices=[[i, 0] for i in range(len(data[k][first:last]))],
            values=data[k][first:last],
            shape=[len(data[k][first:last]),1])
    
    label = tf.constant(labels[first:last])
    return categorical_cols, label

def predict(model,data,labels):
    return model.predict(input_fn=lambda: input_fn(data,labels), as_iterable=False)

In [ ]:
def RTB_simulation(CTR_predictions,bid_predictions,targets,start_budget = 6250000):
    impressions = 0
    clicks = 0
    budget=start_budget
    total_bids = 0
    bids_won = 0
    spent = 0

    for i in range(0,CTR_predictions.shape[0]):
        if CTR_predictions[i][0] > CTR_predictions[i][1]: #Do not bid
            continue
        else:
            current_bid = bid_predictions[i]
            # Check if we still have budget:
            if budget > current_bid:
                # Get the market price:
                payprice = targets[i][1]
                # Check if we win the bid:
                total_bids += 1
                if current_bid > payprice:
                    bids_won += 1
                    impressions += 1
                    budget -= payprice
                    spent += payprice
                    # Check if the person clicks:
                    if targets[i][0] == "1":
                        clicks += 1
            else:
                break
    
    ctr = 0
    avgCPM = 0
    avgCPC = 0

    if clicks > 0:
        avgCPC = spent/clicks
    if impressions > 0:
        avgCPM = spent/impressions
        ctr = (clicks/impressions)*100

    print("Total bids made: " + str(total_bids))
    print("Impressions:{0}".format(impressions))
    print("Percent of bids won: " + str(impressions*100/total_bids))
    print("Clicks:{0}".format(clicks))
    print("avgCPM:" + str(avgCPM))
    print("avgCPC:" + str(avgCPC))
    print("Spent:" + str(spent))
    return ctr

In [ ]:
print("Loading data...")
validation_data = load_data(DL_validation_data_path)
validation_payprice_labels = load_data(DL_validation_payprice_labels_path)
targets = load_data(validation_targets_path)
ctr_training_data = load_data(ctr_training_data_path)
ctr_training_labels = load_data(ctr_training_labels_path)
ctr_validation_data = load_data(ctr_validation_data_path)
print("Done")

In [ ]:
print("Training CTR_estimator")
ctr_estimator = lr_train(ctr_training_data,ctr_training_labels)
print("Done")

In [ ]:
print("Restoring payprice_estimator")
payprice_estimator = restore_model(model_dir_1,dummy_data,model_type="payprice_estimator")
print("Done")

In [ ]:
print("Evaluating CTR_estimator")
ctr_results = lr_evaluation(ctr_estimator,ctr_validation_data)
print("Done")

In [ ]:
print("Evaluating payprice_estimator")
payprice_results = predict(payprice_estimator,validation_data,validation_payprice_labels)
print("Done")

In [ ]:
print("Starting simulation")
CTR = RTB_simulation(ctr_results,payprice_results,targets)

In [ ]:
def data_dump_test(CTR_predictions,bid_predictions,targets,start_budget = 6250000):
    impressions = 0
    clicks = 0
    budget=start_budget
    total_bids = 0
    bids_won = 0
    i = -1
    results = [("bidid","bidprice","advertiser", "pCTR")]
    advertisers = {}
    
    with open(test_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')  
        next(reader)
        for row in reader:
            i += 1
            adver = row[21]
            if adver not in advertisers.keys():
                advertisers[adver] = {'impressions':0,'clicks':0,'ctr':0,'estimated 0s':0,
                                      'estimated 1s':0}
            
            if CTR_predictions[i][0] > CTR_predictions[i][1]: #Do not bid
                advertisers[adver]['estimated 0s'] = advertisers[adver]['estimated 0s'] + 1
                results.append((row[2],0,adver,CTR_predictions[i][1]))
                continue
            else:
                current_bid = bid_predictions[i]
                results.append((row[2],current_bid,adver,CTR_predictions[i][1]))

    return results,advertisers

In [ ]:
print("Starting...")
to_csv,per_advertiser = data_dump_test(ctr_results,payprice_results,targets)
df = pd.DataFrame(to_csv)
df.to_csv(output_path)